# Import statements

In [1]:
from Bio import SeqIO
import pandas as pd
from scripts import utils
from Bio import Entrez
from more_itertools import chunked
from tqdm import tqdm
import pickle
import io

Entrez.email = "gernel@informatik.uni-freiburg.de"

# Load NCBI (protein) entries from RNAInter

In [ ]:
utils.show_rna_inter_protein_databases()

In [ ]:
rna_inter_df = utils.load_rna_inter_protein('NCBI')

In [ ]:
rna_inter_df.to_parquet('rna_inter_protein.parquet', engine='pyarrow', compression=None)

In [6]:
rna_inter_df = pd.read_parquet('results/rna_inter_protein.parquet', engine='pyarrow')
print(f"size of {rna_inter_df.shape[0]:,}")

size of 39,009


In [ ]:
del rna_inter_df

# Obtaining unique protein IDs

In [ ]:
gene_ids = pd.DataFrame(rna_inter_df['Raw_ID2'].unique(), columns=['Raw_ID2'])
gene_ids.to_parquet('rna_inter_protein.parquet', engine='pyarrow', compression=None)

In [ ]:
gene_ids = pd.read_parquet('rna_inter_protein.parquet', engine='pyarrow')

In [ ]:
gene_ids = list(gene_ids['Raw_ID2'])
gene_ids = [gene_id[5:] for gene_id in gene_ids]
print(f"Number of unique proteins: {len(gene_ids)}")

In [ ]:
def fetch_ncbi_nuccore_refseqrna_ids(ids: list):
    handle = Entrez.elink(dbfrom="gene", id=ids, linkname='gene_protein_refseq')
    records = Entrez.read(handle)
    temp_results = []
    for record in records:
        if record['IdList'] == ['0', '0']:
            # This entry does not exist. Therefore, we skipped it.
            continue
        assert len(record['IdList']) == 1
        gene_id = record['IdList'][0]
        for link in record['LinkSetDb']:
            if link['LinkName'] != 'gene_protein_refseq':
                continue
            temp_results += [dict(Raw_ID2=gene_id, Sequence_2_ID=seq['Id']) for seq in link['Link']]
    return temp_results

In [ ]:
def fetch_ncbi_protein_sequences(ids: list):
    handle = Entrez.efetch(db="protein", id=ids, rettype="gb", retmode='xml')
    # Read the protein sequences into a list
    records = Entrez.read(handle)
    return [dict(
            Sequence_2=record.get('GBSeq_sequence'),
            Sequence_2_len=len(record.get('GBSeq_sequence')),
            # Sequence_1_ID=record.get('GBSeq_locus'),
            Sequence_2_ID=record.get('GBSeq_other-seqids')[1][3:]
        ) for record in records]

In [ ]:
count = 0
count_2 = 0
for gene_id in gene_ids:
    gene_id = gene_id[5:]
    protein_ids = fetch_ncbi_nuccore_refseqrna_ids(gene_id)
    print(" ".join([refseq['Sequence_1_ID_2'] for refseq in protein_ids]))
    print(len(protein_ids))
    for protein_id in protein_ids:
        count_2 += 1
        protein_id = protein_id['Sequence_1_ID_2']
        uniprot_ids = utils.fetch_ncbi_uniprot_ids(protein_id)
        if len(uniprot_ids) != 0:
            count += 1
            print(f"{count}/{count_2}")
            break

In [ ]:
protein_ids_df = utils.call_fetch_function(fetch_ncbi_nuccore_refseqrna_ids, 250, gene_ids)
protein_ids_df.to_parquet('ncbi_proteins_ids.parquet', engine='pyarrow')

In [ ]:
# protein_ids_df.to_parquet('ncbi_proteins_ids.parquet', engine='pyarrow')
protein_ids_df = pd.read_parquet('ncbi_proteins_ids.parquet', engine='pyarrow')
protein_ids = list(protein_ids_df['Sequence_2_ID'].unique())

In [ ]:
## Get protein sequences
protein_sequences = utils.call_fetch_function(fetch_ncbi_protein_sequences, 1000, protein_ids)
protein_sequences.to_parquet('ncbi_protein_sequences.parquet', engine='pyarrow')

In [ ]:
protein_sequences = pd.read_parquet('ncbi_protein_sequences.parquet', engine='pyarrow')

In [ ]:
# Join sequences with corresponding ids
ncbi_protein_df = protein_ids_df.merge(protein_sequences, on='Sequence_2_ID', how='inner')
ncbi_protein_df['Raw_ID2'] = "NCBI:" + ncbi_protein_df['Raw_ID2'].astype(str)
ncbi_protein_df.to_parquet('ncbi_proteins.parquet', engine='pyarrow')

In [7]:
ncbi_protein_df = pd.read_parquet('results/ncbi_proteins.parquet', engine='pyarrow')
pass

# Calc recovery rate

In [8]:
utils.calc_recovery_rate(rna_inter_df, ncbi_protein_df, col_name='Raw_ID2')

Unique Gene IDs before extraction:	39,009
Unique Gene IDs after extraction:	38,476
Extraction rate:	98.63%


# Obsolete Code

In [ ]:
def get_ids(ids):
    for _ in range(3):
        try:
            handle = Entrez.efetch(db='gene', id=ids, retmode="xml")
            return Entrez.read(handle)
        except Exception as e:
            print(e)
            continue
    print("Something went 3 times wrong :(!")

In [ ]:
uniport_ids = list()
split_ids = list(chunked(protein_ids, 500))
# handle = Entrez.elink(dbfrom='gene', id=protein_id, db='refseq')
error_ids = []
for ids in tqdm(split_ids):
    records = get_ids(ids)
    if records is None:
        error_ids.append(ids)
        continue
    for record in records:
        comments = record.get('Entrezgene_comments')
        gene_id = record.get('Entrezgene_track-info').get('Gene-track').get('Gene-track_geneid')
        for comment in comments:
            if comment.get('Gene-commentary_heading') != 'NCBI Reference Sequences (RefSeq)':
                continue
            if comment.get('Gene-commentary_comment') is None:
                continue
            for commenary_commment in comment.get('Gene-commentary_comment'):
                if commenary_commment.get('Gene-commentary_products') is None:
                    continue
                for product in commenary_commment.get('Gene-commentary_products'):
                    if product.get('Gene-commentary_products') is None:
                        continue
                    for product_2 in product.get('Gene-commentary_products'):
                        if product_2.get('Gene-commentary_comment') is None:
                            continue
                        for comment_2 in product_2.get('Gene-commentary_comment'):
                            if comment_2.get('Gene-commentary_heading') != 'UniProtKB':
                                continue
                            for comment_3 in comment_2.get('Gene-commentary_comment'):
                                if comment_3.get('Gene-commentary_source') is None:
                                    continue
                                for product_3 in comment_3.get('Gene-commentary_source'):
                                    db = product_3.get('Other-source_src').get('Dbtag').get('Dbtag_db')
                                    object_id = product_3.get('Other-source_src').get('Dbtag').get('Dbtag_tag').get('Object-id').get('Object-id_str')
                                    uniport_ids.append(dict(
                                        gene_id=gene_id,
                                        db_type=db,
                                        object_id=object_id
                                    ))
uniport_ids_df = pd.DataFrame(uniport_ids)
uniport_ids_df.to_parquet('uniport_protein_ids.parquet', engine='pyarrow', compression=None)

In [ ]:
uniport_ids_df = pd.read_parquet('uniport_protein_ids.parquet', engine='pyarrow')
uniport_ids_df = uniport_ids_df.drop_duplicates(subset=None, keep="first", inplace=False)

In [ ]:
object_ids = list(uniport_ids_df['object_id'].unique())
proteins = utils.fetch_protein_fasta(object_ids)

file = open('proteins.pickle', 'wb')
pickle.dump(proteins, file)
file.close()

In [ ]:
file = open('proteins.pickle', 'rb')
proteins = pickle.load(file)
file.close()

In [ ]:
# convert fasta string to Seq Objects
proteins = [(protein[0], list(SeqIO.parse(io.StringIO(protein[1]), "fasta"))) for protein in proteins]
# flatten list
proteins = [(protein[0], seq) for protein in proteins for seq in protein[1]]

In [ ]:
uniprot_proteins_df = [dict(
    object_id=f"{protein[0]}",
    protein_sequence=str(protein[1].seq),
    protein_sequence_len=len(str(protein[1].seq)),
    protein_id=protein[1].id,
    protein_description=protein[1].description
) for protein in proteins]
uniprot_proteins_df = pd.DataFrame(uniprot_proteins_df)
uniprot_proteins_df = pd.merge(uniprot_proteins_df, uniport_ids_df, left_on='object_id', right_on='object_id')
uniprot_proteins_df['gene_id'] = "NCBI:" + uniprot_proteins_df['gene_id'].astype(str)
uniprot_proteins_df = uniprot_proteins_df.drop(columns=['db_type', 'object_id'])
uniprot_proteins_df.to_parquet("protein_ncbi.parquet", engine='pyarrow', compression=None)

# Load UniProt IDs which are stored directly in RNAInter database

In [ ]:
uniport_ids_2_df = utils.load_rna_inter_protein('UniProt')
uniport_ids_2_df.to_parquet('RNAInter_uniport.parquet', engine='pyarrow', compression=None)

In [ ]:
uniport_ids_2_df = pd.read_parquet('RNAInter_uniport.parquet', engine='pyarrow')

In [ ]:
object_ids = list(uniport_ids_2_df['Protein_ID'].unique())
proteins = utils.fetch_protein_fasta(object_ids)
# convert fasta string to Seq Objects
proteins = [(protein[0], list(SeqIO.parse(io.StringIO(protein[1]), "fasta"))) for protein in proteins]
# flatten list
proteins = [(protein[0], seq) for protein in proteins for seq in protein[1]]
file = open('proteins_2.pickle', 'wb')
pickle.dump(proteins, file)
file.close()

In [ ]:
file = open('proteins_2.pickle', 'rb')
proteins = pickle.load(file)
file.close()

In [ ]:
uniprot_proteins_2_df = [dict(
    gene_id=f"UniProt:{protein[0]}",
    protein_sequence=str(protein[1].seq),
    protein_sequence_len=len(str(protein[1].seq)),
    protein_id=protein[1].id,
    protein_description=protein[1].description
) for protein in proteins]
uniprot_proteins_2_df = pd.DataFrame(uniprot_proteins_2_df)
uniprot_proteins_2_df.to_parquet("protein_uniprot.parquet", engine='pyarrow', compression=None)

In [ ]:
uniprot_proteins_2_df = pd.read_parquet("protein_uniprot.parquet", engine='pyarrow')

In [ ]:
# uniprot_proteins_df = uniprot_proteins_df[uniprot_proteins_df['protein_sequence_len'] < 2000]
utils.analyze_protein_sequence_lens([500, 800, 1000, 2000, 5000, 10000], uniprot_proteins_all_df)

# Concat both UniProt Sequences together

In [ ]:
uniprot_proteins_all_df = pd.concat([uniprot_proteins_df, uniprot_proteins_2_df])
uniprot_proteins_all_df.to_parquet("protein_rna_inter.parquet", engine='pyarrow', compression=None)

In [ ]:
uniprot_proteins_all_df = pd.read_parquet("protein_rna_inter.parquet", engine='pyarrow')

# Read Protein Sequences from ESM-Model (esm2_t33_650M_UR50D)
## Properties: 33 layers, 650 million parameters, embedding dim 1280
Note: This model takes around 9GB out of 12GB of available video memory (RTX 2080). However, protein sequences up to a length of 500 take up around 2GB of video memory. Therefore, longer sequences (& all other sequences) will be embedded with a smaller model as well.

In [ ]:
file = open('protein_sequence_embeddings_esm2_t33_650M_UR50D.pickle', 'rb')
protein_embeddings_big = pickle.load(file)
file.close()
# Protein Embeddings Big DataFrame
peb_df = pd.DataFrame([dict(
    protein_sequence=embd[0],
    protein_embedding_big=embd[1]
) for embd in protein_embeddings_big])
peb_df.to_parquet('protein_sequence_embeddings_esm2_t33_650M_UR50D.parquet', engine='pyarrow', compression=None)

In [ ]:
peb_df = pd.read_parquet('protein_sequence_embeddings_esm2_t33_650M_UR50D.parquet', engine='pyarrow')

In [ ]:
pass